In [1]:
!pip install word2number

In [2]:
# Core

import pandas as pd
# Configuración para que muestre todas las columnas al completo
pd.set_option('display.max_columns', None)
# Configuración para que no muestre la notación científica
pd.set_option('display.float_format', '{:,.0f}'.format)

# Configuración

import warnings
warnings.filterwarnings("ignore")

# Utilidades

# para convertir str en números
from word2number import w2n

In [3]:

def apertura_exploracion(csv):

    """ Función para leer csv, convertir a df y hacer una primera exploración.
    Igualar a variable con el nombre que quieres dar a DataFrame"""
    
    try:
        # Convertir el csv a DataFrame
        df = pd.read_csv(f"../data/{csv}.csv")        

        # Muestro las primeras filas
        display(df.head())

        # Muestro las últimas filas
        display(df.tail())

        # Muestro las dimensiones del dataframe
        print(f"-----\n\nEl DataFrame tiene {df.shape[0]} filas y {df.shape[1]} columnas.\n-----")

        # Consulto si hay filas duplicadas
        print(f"\nEl número de filas duplicadas es {df.duplicated().sum()}\n-----")

        # Muestro el tipo de dato y si hay nulos por cada columna
        print("\nInformación del DataFrame:")
        df.info()

        # Muestro el porcentaje de nulos por variable
        print("\nPorcentaje de nulos:")
        display(round(df.isnull().mean() * 100, 2))

        # Muestro las estadísticas de columnas numéricas
        print("-----\n\nEstadísticas descriptivas:")
        display(df.describe(include="all").T)

        # Me devuelve un df que tendré que igualar a una variable
        return df  
                

    # Excepciones en caso de no encontrar el archivo o de que haya un error
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '../data/{csv}.csv'.")
        return None  
    
    except Exception as e:
        print(f"Ocurrió un error: {e}")
        return None 

In [4]:
df_movies = apertura_exploracion("raw_movies_dataset")

,Title,Genre,Year,Budget,Revenue,IMDB_Rating
0,The Last Journey,Drama,2020,11000000,"34,049,694",5
1,Infinite Dreams,Drama,2004,178000000,"550,436,005",7
2,Code of Shadows,Romance,2018,13000000,"7,855,773",5
3,Ocean Whisper,Action,2016,148000000,"176,997,903",7
4,Forgotten Realm,Romance,2017,61000000,"126,406,131",NaN


,Title,Genre,Year,Budget,Revenue,IMDB_Rating
17,Shattered Skies,NaN,2008,59000000,"218,062,610",7
18,Eternal Voyage,Fantasy,2022,59000000,NaN,6
19,Mystic River,NaN,2020,41000000,"58,510,304",5
20,The Last Journey,Drama,2020,11000000,"34,049,694",5
21,Shattered Skies,NaN,2008,59000000,"218,062,610",7


-----

El DataFrame tiene 22 filas y 6 columnas.
-----

El número de filas duplicadas es 2
-----

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        22 non-null     object 
 1   Genre        18 non-null     object 
 2   Year         22 non-null     object 
 3   Budget       22 non-null     object 
 4   Revenue      19 non-null     float64
 5   IMDB_Rating  18 non-null     float64
dtypes: float64(2), object(4)
memory usage: 1.2+ KB

Porcentaje de nulos:


Title          0
Genre         18
Year           0
Budget         0
Revenue       14
IMDB_Rating   18
dtype: float64

-----

Estadísticas descriptivas:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Title,22,20,The Last Journey,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Genre,18,7,Drama,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Year,22,15,2020,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Budget,22,19,59000000,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Revenue,19,NaN,NaN,NaN,"173,447,916","136,688,547","7,855,773","59,656,930","150,236,576","221,459,089","550,436,005"
IMDB_Rating,18,NaN,NaN,NaN,6,1,4,5,6,7,9


In [5]:
# Miro los valores que se encuentran dentro de la variable "Title":
#   - encuentro los que están duplicados
#   - compruebo si es necesaria la normalización del texto, NO

df_movies["Title"].value_counts()

Title
The Last Journey        2
Shattered Skies         2
Infinite Dreams         1
Eternal Voyage          1
Broken Destiny          1
Beyond the Stars        1
The Lost Signal         1
Crimson Night           1
Quantum Hearts          1
Solar Storm             1
Iron Legacy             1
Whispers in the Dark    1
Digital Frontier        1
Echoes of Tomorrow      1
Silent Horizon          1
Neon City               1
Forgotten Realm         1
Ocean Whisper           1
Code of Shadows         1
Mystic River            1
Name: count, dtype: int64

In [6]:
# Miro las filas duplicadas con sus originales, compruebo que son realmente duplicados.

df_movies[df_movies.duplicated(keep=False)].sort_values("Title")

,Title,Genre,Year,Budget,Revenue,IMDB_Rating
17,Shattered Skies,NaN,2008,59000000,"218,062,610",7
21,Shattered Skies,NaN,2008,59000000,"218,062,610",7
0,The Last Journey,Drama,2020,11000000,"34,049,694",5
20,The Last Journey,Drama,2020,11000000,"34,049,694",5


In [7]:
# Elimino los duplicados.

df_movies.drop_duplicates(inplace=True)

In [8]:
# Compruebo las nuevas dimensiones una vez eliminados los duplicados.

df_movies.shape

(20, 6)

In [9]:
# Compruebo los valores que se encuentran dentro de la variable "Year"
# Busco una librería para convertir str a numero, siempre y cuando no hayan errores ortográficos ni de puntuación.

df_movies["Year"].value_counts()

Year
2020            3
2011            2
2007            2
2022            2
2004            1
2018            1
2016            1
2017            1
2000            1
2006            1
2003            1
2019            1
Two Thousand    1
2001            1
2008            1
Name: count, dtype: int64

In [29]:
def text_to_num(year):
    """
    Limpiar y convertir a números str sin errores ortográficos ni de puntuación: 

        - Si el valor es un str, lo normaliza dejando todo en minusculas y quitando espacios en los extremos.
        - Después lo convierte a número con la librería w2n, método word_to_num
        - Si no puede devuelve el valor ya normalizado
        - Si no es un str, devuelve el valor original
    """
    # Si es str
    if isinstance(year, str):
        # Normalizamos el str
        year_limpio = year.strip().lower()
        try:
            return w2n.word_to_num(year_limpio)
        except:
            return year_limpio
    # Si no es str, devuelve el mismo valor   
    else: 
        return year

In [30]:
# Aplico la función a la variable "Year" del DataFrame

df_movies["Year"] = df_movies["Year"].apply(text_to_num)

In [31]:
# Compruebo el resultado, ordenando el output por año

df_movies["Year"].value_counts().sort_index()

Year
2000    2
2001    1
2003    1
2004    1
2006    1
2007    2
2008    1
2011    2
2016    1
2017    1
2018    1
2019    1
2020    3
2022    2
Name: count, dtype: int64